In [14]:
import import_ipynb
import math

class Data_Processor:
    
    
    #initializes any starting data members
    def __init__(self):
        #raw_date_time_array contain dates and times, raw_glucose_level_array contains glucose level
        self.raw_date_time_array = []
        self.raw_glucose_level_array = [] 

        self.complete_time_set = [[]] #this array holds all 24 hour time cycles; it is a 2 dimensional array
        # - each index represents a day (24 hour cycle), contains an array
        # - each index of 2nd array is a time cycle
        self.complete_glucose_set = [[]] #this array holds all glucose values. Each index is 24 hr cycle array containing different glucose values
        self.complete_set_index = 0 #This represents the current index in the complete_set arrays 


        #these contain small amounts of data that will be loaded into the complete time set
        self.short_current_date = []
        self.possible_next_day = []
        pass



    #_____________________________________________________________________________________________________________________________________
    #1. Stores all raw time and glucose data into separate arrays
    def Read_Raw_Data(self):
        import xml.etree.ElementTree as ET

        tree = ET.parse('/Users/niharkoppolu/Desktop/OHIOT1D/540-ws-training.xml')

        root = tree.getroot()

        
        for data_point in root[0]:
            data_point_dict = data_point.attrib#each attribute is a dictionary
            self.raw_date_time_array.append(data_point_dict.get("ts"))
            self.raw_glucose_level_array.append(int(data_point_dict.get("value")))
            pass

        #Check to see if data correctly read in 
        #print(data_time_array[0], data_time_array[len(data_time_array) - 1])
        #print(self.raw_glucose_level_array[0])
        pass



    #_____________________________________________________________________________________________________________________________________
    #2. Converts time into 24 hour cycle and converts glucose to integer, Inserts any missing data values into time and glucose arrays
    def Process_Raw_Data(self):

        set_size = len(self.raw_date_time_array)
        

        #print(self.raw_date_time_array[0])#19-05-2027 11:36:29
        previous_split_set = self.raw_date_time_array[0].split(' ')
        prev_glucose = self.raw_glucose_level_array[0]
        index = 1
        #This loop sends processes and adds each set of glucose and time values one at at time to the complete_sets iteratively 
        while index < set_size:
            current_split_set = self.raw_date_time_array[index].split(' ')
            newest_glucose = self.raw_glucose_level_array[index]
            
            #NEW_DATE_FUNC call to generate new short_time_arrays
            new_day_flag = self.__new_date_func(current_split_set, previous_split_set)
            #ADD_TO_COMPLETE_SETS call to add short_time_array content to complete_set arrays
            self.__add_to_complete_sets(new_day_flag, prev_glucose, newest_glucose)

            previous_split_set = current_split_set
            prev_glucose = newest_glucose
            index += 1
            pass
        #"""
        pass
    


    #______________________________________________________________________________________________________________________________________
    #3. Adds data to complete_data_sets from the short_time_arrays and also fills in corresponding values in the glucose_array
    def __add_to_complete_sets(self, new_day_flag, prev_glucose, newest_glucose):
        
        #the if statement gets rid of copies in both complete_sets  
        last_value = len(self.complete_time_set[self.complete_set_index]) - 1
        if (last_value > -1 and (self.short_current_date[0] == self.complete_time_set[self.complete_set_index][last_value])):     
            del(self.complete_time_set[self.complete_set_index][-1])
            del(self.complete_glucose_set[self.complete_set_index][-1])
            pass


        self.complete_time_set[self.complete_set_index].extend(self.short_current_date)

        #Adding values to the complete_glucose_set 
        self.complete_glucose_set[self.complete_set_index].append(prev_glucose)
        
        no_glucose_to_add = len(self.short_current_date) - 2
        #This loop adds middle glucose values to complete_glucose_array
        i = 1
        while i <= no_glucose_to_add:
            temp = prev_glucose + ((newest_glucose - prev_glucose) * (i / (no_glucose_to_add + 1)))
            self.complete_glucose_set[self.complete_set_index].append(temp)
            i += 1 
            pass

        #if the new day flag is set, then complete_set_index is incremented and values in possible_next_day are added to the complete_time_set
        if (new_day_flag):
            self.complete_set_index += 1
            self.complete_time_set.append(self.possible_next_day)
            self.complete_glucose_set.append([newest_glucose])
            pass
        else:
            self.complete_glucose_set[self.complete_set_index].append(newest_glucose)
        #add final glucose value to complete_set
        

        pass  
    
   
    #______________________________________________________________________________________________________________________________________
    #4. Call func that creates short_time arrays from Prev and Current
    def __new_date_func(self, current, prev):
        #Converting current and previous to date values
        previous_day = prev[0]
        previous_time = prev[1]
        current_day = current[0] 
        current_time = current[1]
    
        prev_exact_date = int(previous_day.split('-')[0])#ex date: 19-05-2027 (19 is the date)
        current_exact_date = int(current_day.split('-')[0]) 


        #Checking if dates are the same or different
        if prev_exact_date == current_exact_date:
            new_date_flag = False
        else:
            new_date_flag = True
            #set day_seg then return it
        
        #Call to Time Process function 
        self.short_current_date, self.possible_next_day = self.__time_process_func(previous_time, current_time, new_date_flag)



        return new_date_flag
        pass


    #_____________________________________________________________________________________________________________________________________
    #5. Converts previous and current time values into integer values placed in short_time arrays
    def __time_process_func(self, previous_time, current_time, new_date_flag):    
        #storing current and previous minute and sec values
        temp_time_array = previous_time.split(':')
        prev_hour = int(temp_time_array[0])*60 #converted to minute
        prev_minute = int(temp_time_array[1])
        prev_second = int(temp_time_array[2])/60 #converted to fraction of minute
        prev_time = prev_minute + prev_second + prev_hour 
        
        temp_time_array = current_time.split(':')
        current_hour = int(temp_time_array[0])*60 #converted to minute
        current_minute = int(temp_time_array[1]) 
        current_second = int(temp_time_array[2])/60 #converted to fracton of minute
        current_time = current_minute + current_second + current_hour

        
        #If the time is within the 5 time mark
        #may be used if new_day_flag is sef if ((current_time <= prev_time + 5 and prev_time + 5 <= 60) or (prev_time + 5 > 60 and current_time + 60 <= prev_time + 5)):
        
        #If the new_date_flag is set to false
        if(new_date_flag == False):
            possible_next_day = []
            short_current_date = [prev_time] #holds new time set
            #
            if (current_time <= prev_time + 5):
                pass
            
            #If the time is beyond the 5 time mark, append to the middle of the short_time array
            else:
                no_added_data_points = math.ceil((current_time - prev_time)/5)
                dividing_factor = no_added_data_points + 1

                for i in range(no_added_data_points):
                    short_current_date.append(prev_time +((current_time - prev_time)* ((i + 1) / dividing_factor)))
                    pass

            short_current_date.append(current_time)

        #if new_date_flag set to true start next day 24 hour cycle exactly where the current time starts, don't add any more times between
        if(new_date_flag == True):
            short_current_date = [prev_time]
            possible_next_day = [current_time]
            pass

        return short_current_date, possible_next_day

    
    #_____________________________________________________________________________________________________________________________________
    #6. USED TO TEST PROGRAM: Graphs and shows data associated with a single day of CGM data   
    def Graph_Single_Date(self):
        import matplotlib.pyplot as plt

        day = 2

        no_points_in_day = len(self.complete_time_set[day])

        #scaling x axis to hours
        x_axis = []
        for i in range(no_points_in_day):
            x_axis.append(self.complete_time_set[day][i]/60)
            pass

        y_axis = self.complete_glucose_set[day]

        

        print("day time:", self.complete_time_set[day][150]/60)
        print("glucose level:", self.complete_glucose_set[day][150])
        print("# points: ", no_points_in_day)
        plt.plot(x_axis, y_axis)
        plt.show()
        pass
        
